## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Kodiak,US,57.79,-152.41,60.80,72,20,6.73,few clouds
1,1,Maxixe,MZ,-23.86,35.35,59.95,81,0,0.67,clear sky
2,2,Puerto Ayora,EC,-0.74,-90.35,64.99,97,77,8.72,broken clouds
3,3,Yumen,CN,40.28,97.20,80.85,22,0,8.50,clear sky
4,4,Nelson Bay,AU,-32.72,152.15,62.60,36,0,26.40,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3,3,Yumen,CN,40.28,97.20,80.85,22,0,8.50,clear sky
8,8,Tirumullaivasal,IN,11.23,79.83,86.16,69,87,11.39,moderate rain
11,11,Butaritari,KI,3.07,172.79,83.08,67,98,7.90,overcast clouds
14,14,Vaini,TO,-21.20,-175.20,80.60,83,20,11.41,few clouds
17,17,Jieshi,CN,22.81,115.83,85.35,67,41,5.17,scattered clouds
19,19,Maria Chiquita,PA,9.44,-79.76,77.00,94,20,5.01,few clouds
23,23,Hasaki,JP,35.73,140.83,81.00,88,75,4.70,light intensity shower rain
24,24,Ixtapa,MX,20.70,-105.20,78.01,94,90,11.41,very heavy rain
35,35,Kahului,US,20.89,-156.47,87.80,55,40,18.34,scattered clouds
43,43,Hilo,US,19.73,-155.09,82.40,69,90,6.93,overcast clouds


In [6]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                235
City                   235
Country                235
Lat                    235
Lng                    235
Max Temp               235
Humidity               235
Cloudiness             235
Wind Speed             235
Current Description    235
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna(axis='rows')

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Yumen,CN,80.85,clear sky,40.28,97.20,
8,Tirumullaivasal,IN,86.16,moderate rain,11.23,79.83,
11,Butaritari,KI,83.08,overcast clouds,3.07,172.79,
14,Vaini,TO,80.60,few clouds,-21.20,-175.20,
17,Jieshi,CN,85.35,scattered clouds,22.81,115.83,
19,Maria Chiquita,PA,77.00,few clouds,9.44,-79.76,
23,Hasaki,JP,81.00,light intensity shower rain,35.73,140.83,
24,Ixtapa,MX,78.01,very heavy rain,20.70,-105.20,
35,Kahului,US,87.80,scattered clouds,20.89,-156.47,
43,Hilo,US,82.40,overcast clouds,19.73,-155.09,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"
    
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [10]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.loc[(hotel_df["Hotel Name"] == "")] = np.nan
clean_hotel_df = hotel_df.dropna(axis='rows')
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
8,Tirumullaivasal,IN,86.16,moderate rain,11.23,79.83,"MATHAMPATTINAM, MODUNUTTI GROUP"
11,Butaritari,KI,83.08,overcast clouds,3.07,172.79,Isles Sunset Lodge
14,Vaini,TO,80.60,few clouds,-21.20,-175.20,Keleti Beach Resort
17,Jieshi,CN,85.35,scattered clouds,22.81,115.83,Yuehai Hotel
19,Maria Chiquita,PA,77.00,few clouds,9.44,-79.76,La Casa del Rio
23,Hasaki,JP,81.00,light intensity shower rain,35.73,140.83,Inubosaki Kanko Hotel
24,Ixtapa,MX,78.01,very heavy rain,20.70,-105.20,Container Inn Puerto Vallarta
35,Kahului,US,87.80,scattered clouds,20.89,-156.47,Maui Seaside Hotel
43,Hilo,US,82.40,overcast clouds,19.73,-155.09,Hilo Hawaiian Hotel
46,Erzin,TR,79.00,clear sky,36.96,36.20,Artemis Otel


In [11]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [13]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))